In [1]:
#pip install unidecode

In [2]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import unidecode
import pandas as pd

Здесь надо ввести текст для поиска вакансии

In [3]:
text= 'python'

In [4]:
main_link = 'https://hh.ru'
params = {'L_save_area':'true',
          'clusters':'true',
         'enable_snippets':'true',
         'showClusters':'true'}
page = '0'


In [5]:
params['text'] = text
params['page'] = page

In [6]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}

In [7]:
response = requests.get(main_link+'/search/vacancy', params=params, headers=headers)

In [8]:
vacancies=[]
while True:
    next_page=None
    if response.ok:
        soup = bs(response.text, 'html.parser')
        next_page = soup.find('a',{'data-qa':"pager-next"})
        vacancies_list = soup.findAll('div', {'class':'vacancy-serp-item'})
        
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            vacancy_all = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'})
            vacancy_link = vacancy_all['href']
            vacancy_name = vacancy_all.getText()
            vacancy_site = "hh.ru"
            vacancy_comp = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            if (vacancy_comp!=None):
                #обрабатываем строку с зарплатой
                salary_str = vacancy_comp.getText()
                a = salary_str.split(' ')
                currency = a[-1]
                #смотрим - указан ли в зп диапазон
                salary_number = a[-2].split('-')
                #если да - то обрабатываем ситуацию, когда есть и максимальная, и минимальная зп
                if (len(salary_number)==2):
                    salary_min = unidecode.unidecode(salary_number[0])
                    #salary_max = re.sub('\xa',' ',salary_number[1])
                    salary_max = unidecode.unidecode(salary_number[1])
                #если нет - то обрабатываем ситуацию, когда есть только минимальная зп
                else: 
                    if a[0]=='от':
                        salary_min=unidecode.unidecode(salary_number[0])
                        salary_max=None
                        #или когда есть только максимальная зп
                    else:
                        if a[1]=='до':   
                            salary_min=None
                            salary_max=unidecode.unidecode(salary_number[1])

            else: 
                salary_max=None
                salary_min=None
                currency= None
            
            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = vacancy_link
            vacancy_data['from_site'] = vacancy_site
            vacancy_data['min_compensation'] = salary_min
            vacancy_data['max_compensation'] = salary_max
            vacancy_data['currency'] = currency
            vacancies.append(vacancy_data)
            
        if (next_page!=None):
            next_page_link = main_link+next_page['href']
            response = requests.get(next_page_link, headers=headers)
        else: break

In [9]:
vacancies



[{'name': 'Python Developer',
  'link': 'https://samara.hh.ru/vacancy/43048662?query=python',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'currency': None},
 {'name': 'Python Developer',
  'link': 'https://samara.hh.ru/vacancy/43046900?query=python',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'currency': None},
 {'name': 'Data Analyst',
  'link': 'https://samara.hh.ru/vacancy/43075693?query=python',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'currency': None},
 {'name': 'Quantitative developer (HFT developer)',
  'link': 'https://samara.hh.ru/vacancy/43005027?query=python',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'currency': 'руб.'},
 {'name': 'Quantitative developer (HFT developer)',
  'link': 'https://samara.hh.ru/vacancy/43005025?query=python',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'c

In [10]:
df = pd.DataFrame(vacancies)

In [11]:
df

,name,link,from_site,min_compensation,max_compensation,currency
0,Python Developer,https://samara.hh.ru/vacancy/43048662?query=py...,hh.ru,None,None,None
1,Python Developer,https://samara.hh.ru/vacancy/43046900?query=py...,hh.ru,None,None,None
2,Data Analyst,https://samara.hh.ru/vacancy/43075693?query=py...,hh.ru,None,None,None
3,Quantitative developer (HFT developer),https://samara.hh.ru/vacancy/43005027?query=py...,hh.ru,None,None,руб.
4,Quantitative developer (HFT developer),https://samara.hh.ru/vacancy/43005025?query=py...,hh.ru,None,None,руб.
...,...,...,...,...,...,...
1995,Python developer Банк 131,https://samara.hh.ru/vacancy/42629229?query=py...,hh.ru,None,None,None
1996,Python developer (Multiscaner),https://samara.hh.ru/vacancy/41906395?query=py...,hh.ru,None,None,None
1997,Automation QA (Python),https://samara.hh.ru/vacancy/41965926?query=py...,hh.ru,None,None,None
1998,Python Django Web Developer,https://samara.hh.ru/vacancy/42775108?query=py...,hh.ru,None,None,None
